In [ ]:
!sudo apt-get install portaudio19-dev python-pyaudio
!sudo apt-get install python-gnuradio-audio-portaudio
!pip install PyAudio
!pip install SpeechRecognition
!pip install pyttsx3
!pip install wavio
!pip install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev python-pyaudio
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 208 kB of archives.
After this operation, 999 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-pyaudio amd64 0.2.11-1build2 [24.1 kB]
Fetched 208 kB in 0s (2,748 kB/s)
debconf: unable to initialize frontend: Di

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python-gnuradio-audio-portaudio


  Created wheel for PyAudio: filename=PyAudio-0.2.11-cp37-cp37m-linux_x86_64.whl size=52572 sha256=3e1c5fc4e20085d4e4bfa18aae21810f0afc878e9f625ce4caa22874867ec1ef
  Stored in directory: /root/.cache/pip/wheels/f4/a8/a4/292214166c2917890f85b2f72a8e5f13e1ffa527c4200dcede
Successfully built PyAudio
     |████████████████████████████████| 32.8MB 98kB/s 


In [ ]:
# all imports
import numpy as np
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from pydub import AudioSegment


RECORD = """
const sleep = time => new Promise(resolve => {
setTimeout(resolve, time)
}, )
const b2text = blob => new Promise(resolve => {
const reader = new FileReader()
reader.onloadend = e => resolve(e.srcElement.result)
reader.readAsDataURL(blob)
})
var espacio = document.querySelector("#output-area")
var record = time => new Promise(async resolve => {
stream = await navigator.mediaDevices.getUserMedia({ audio: true })
recorder = new MediaRecorder(stream)
chunks = []
recorder.ondataavailable = e => chunks.push(e.data)
recorder.start()
var numerillo = (time/1000)-1
for (var i = 0; i < numerillo; i++) {
espacio.appendChild(document.createTextNode(numerillo-i))
await sleep(1000)
espacio.removeChild(espacio.lastChild)
}
recorder.onstop = async ()=>{
blob = new Blob(chunks)
text = await b2text(blob)
resolve(text)
}
recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  return 'audio.wav'  # or webm ?

In [ ]:
EXPECTED_SAMPLE_RATE = 16000

def convert_audio_for_model(user_file, output_file='converted_audio_file.wav'):
  audio = AudioSegment.from_file(user_file)
  audio = audio.set_frame_rate(EXPECTED_SAMPLE_RATE).set_channels(1)
  audio.export(output_file, format="wav")
  return output_file

In [ ]:
import speech_recognition as sr

r = sr.Recognizer()
n=int(input("Enter the number of seconds:"))
print("Speak Anything :")
print("Time Remaining:")
wave=record(n)
converted_audio_file = convert_audio_for_model(wave)

# obtain audio from the microphone
r = sr.Recognizer()
with sr.AudioFile('/content/converted_audio_file.wav') as source:
    audio = r.record(source) #read the wave file from the source

# recognize speech using Google Speech Recognition
try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`
    print("You said:  " + r.recognize_google(audio))
except sr.UnknownValueError:
    print("AI could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))

Enter the number of seconds:5
Speak Anything :
Time Remaining:


<IPython.core.display.Javascript object>

You said:  hello am I audible
